In [1]:
import numpy as np
import matplotlib.pyplot as plt
import MARIGOLD as mg
import os
import openpyxl as op

%load_ext autoreload
%autoreload 2

In [2]:
############################################################################################################################
#                                                                                                                          #
#                                                         INPUTS                                                           #
#                                                                                                                          #
############################################################################################################################
author = 'quan'                 # Determines which dataset you're using
refetch = True                  # Refetch data?
xl_compare = False              # When plotting, compare against Excel?
multiplot = False               # When plotting, plot with ai with contributions, void fraction, pressure

z_step = 0.01                   # Axial step size [-] nondimensional, units in L/D ***If you're inputting in [m], convert to diameters
endpoints = [116, 244]
modes = [None, None]

n_ports = 3
n_conditions = 9

passkey = 'True'                # Logical statement which can restrict which conditions to run -- see below for example
# passkey = ('cond.jf == 3.98 and cond.jgref == 0.25')

In [4]:
############################################################################################################################
#                                                                                                                          #
#                                                          DATA                                                            #
#                                                                                                                          #
############################################################################################################################
# Extracts data from database, specified by author
# Will want to change the working directory to where you keep your spreadsheets

database = None

if author == 'ryan':
    cwd = os.path.abspath("H:\TRSL-H\IATE\Data\spreadsheets\\25.4mm_ryan")
    dat_to_fetch = 'database_r.dat'

    if refetch:
        mg.extractLocalDataFromDir(cwd, dump_file=dat_to_fetch, sheet_type='ryan_template')

elif author == 'quan':
    cwd = os.path.abspath("H:\TRSL-H\IATE\Data\spreadsheets\\25.4mm_quan_2")
    dat_to_fetch = 'database_q.dat'

    if refetch:
        mg.extractLocalDataFromDir(cwd, dump_file=dat_to_fetch, sheet_type='adix_template')

elif author == 'yadav':
    cwd = os.path.abspath("H:\TRSL-H\IATE\Data\spreadsheets\\50.8mm")
    dat_to_fetch = 'database_y.dat'

    if refetch:
        mg.extractLocalDataFromDir(cwd, dump_file=dat_to_fetch, sheet_type='yadav_template')

elif author == 'bettis':
    cwd = os.path.abspath("H:\TRSL-H\IATE\Data\Data_Summary_OneGroup")
    dat_to_fetch = 'database_b.dat'

    if refetch:
        mg.extractLocalDataFromDir(cwd, dump_file=dat_to_fetch, sheet_type='bettis_template')

else:
    database = mg.loadProbeData('./database.dat')

if database == None:
    database = mg.loadProbeData(dat_to_fetch)

In [ ]:
############################################################################################################################
#                                                                                                                          #
#                                                     DATA PROCESSING                                                      #
#                                                                                                                          #
############################################################################################################################
# This module does a few things:
# 1. Unique flow condition finding and sorting, for plotting and other hijinks
# 2. Determine starting L/D
# 3. Void profile reconstruction for all conditions except initial condition

LoverDs = []
designations = []

for cond in database:
    cond:mg.Condition

    designations.append([cond.theta, cond.jf, cond.jgref])
    LoverDs.append(cond.LoverD)

conditions = np.unique(np.array(designations),axis=0)
LoverDs = np.unique(np.array(LoverDs)).sort()

########################################################################################################################
# Condition organization

condor = [[0]*n_ports for _ in range(n_conditions)]

for cond in database:
    cond:mg.Condition

    designation = [cond.theta, cond.jf, cond.jgref]

    run_idx = int(np.where((umap == cmap[i]).all(axis=1))[0])
    port_idx = int(np.where((umap == cmap[i]).all(axis=1))[0])

    condor[run_ID-1][port_ID-1] = cond

sp = np.min(LoverDs)
print(f"\nStarting L/D: {sp}")

########################################################################################################################
# Void reconstruction

for cond in database:
    cond:mg.Condition

    cond.mirror(sym90=True)

    if cond.LoverD != sp:                
        cond.reconstruct_void(method='talley',avg_method='legacy_old')


In [ ]:
############################################################################################################################
#                                                                                                                          #
#                                                          IATE                                                            #
#                                                                                                                          #
############################################################################################################################
# This module runs the IATE itself
# Iterates through organized conditions. Input 'passkey' can be used if you want to impose any restrictions on which conditions are being run

for i, cond in enumerate([row[0] for row in condor]):
    cond:mg.Condition

    if eval(passkey):
        print(cond)
        
        cond.mirror(sym90=True)     # Paranoia paranoia

        # Plot preparation
        plt.figure(i,figsize=(7,5))
        plt.suptitle("${}^o$, $j_f$ = {} [m/s], $j_g$ = {} [m/s]".format(int(cond.theta),cond.jf,cond.jgref))

        ai_st = 0
        if cond.LoverD == sp:
            for ii, cp in enumerate(endpoints):

                if ii == 0:
                    io = mg.iate_1d_1g(
                        cond, query = cp, z_step = z_step,
                        restriction = modes[ii],
                        cond2 = condor[i][ii+1]
                        )
                    
                    # Used for labelling legend
                    label_ai = 'ai'
                    label_ti = 'TI'
                    label_rc = 'RC'
                    label_exp = 'EXP'
                    label_we = 'WE'
                    label_vg = 'VG'

                else:       # If not initial condition, use outputs of previous run as new inputs
                    print(cond)
                    io = mg.iate_1d_1g(
                        condor[i][ii], query = cp, z_step = z_step, io = io,
                        restriction = modes[ii],
                        cond2 = condor[i][ii+1]
                        )
                    
                    # Suppress legend labels
                    label_ai = None
                    label_ti = None
                    label_rc = None
                    label_exp = None
                    label_we = None
                    label_vg = None
                
                z_mesh  = io["z_mesh"]
                ai      = io["ai"]
                aiti    = io["aiti"]
                airc    = io["airc"]
                aiexp   = io["aiexp"]
                aiwe    = io["aiwe"]
                aivg    = io["aivg"]
                alpha   = io["alpha"]
                pz      = io["pz"]

                z_mesh = z_mesh / cond.Dh

                # Plotting
                if multiplot:
                    # IAC plot
                    plt.figure(i)
                    plt.subplot(221)
                    plt.plot(z_mesh,ai,'k',linewidth=3)
                    plt.ylabel('$a_i$ [1/m]')
                    plt.xlabel('L/D [-]')

                    # IAC contributions plot, formatting to match MATLAB styles
                    plt.figure(i)
                    plt.subplot(222)
                    plt.plot(z_mesh,ai-ai[0]+ai_st,'k',linewidth=4,label=label_ai)
                    plt.plot(z_mesh,aiti,'b',linestyle=(0,(3,3)),linewidth=2,label=label_ti)
                    plt.plot(z_mesh,-airc,'r',linestyle='dotted',linewidth=4,label=label_rc)
                    plt.plot(z_mesh,aiexp,'aqua',linewidth=2,label=label_exp)
                    plt.plot(z_mesh,-aiwe,'lime',linestyle=(0,(3,1,1,1)),linewidth=2,label=label_we)
                    plt.plot(z_mesh,-aivg,'magenta',linestyle=(0,(1,1)),linewidth=2,label=label_vg)
                    plt.legend(loc='upper right')
                    plt.ylabel(r'$\Delta$$a_i$ [1/m]')
                    plt.xlabel('L/D [-]')
                    
                    # Void fraction plot
                    plt.figure(i)
                    plt.subplot(223)
                    plt.plot(z_mesh,alpha,'k',linewidth=3)
                    plt.ylabel(r'$\alpha$ [-]')
                    plt.xlabel('L/D [-]')

                    # Pressure plot
                    plt.figure(i)
                    plt.subplot(224)
                    plt.plot(z_mesh,pz,'k',linewidth=3)
                    plt.ylabel('$p_{z}$ [Pa]')
                    plt.xlabel('L/D [-]')
                    
                else:
                    # IAC plot only
                    plt.figure(i)
                    plt.plot(z_mesh,ai,'k',linewidth=4)
                    plt.ylabel('$a_i$ [1/m]')
                    plt.xlabel('L/D [-]')

                ai_st = ai_st + ai[-1] - ai[0]

############################################################################################################################
#                                                                                                                          #
#                                                    PLOT COMPARISONS                                                      #
#                                                                                                                          #
############################################################################################################################
# Plots data (can plot MG data or sheet area-averages, if available)
# Plots previous MATLAB runs, if available

for i, row in enumerate([row for row in condor]):
    for ii, cond in enumerate([col for col in row]):
        cond:mg.Condition
        
        if eval(passkey):

            perr = 0.1

            if multiplot:
                plt.figure(i)
                plt.subplot(221)
                plt.errorbar(cond.LoverD,cond.area_avg('ai',method='legacy_old'),cond.area_avg('ai',method='legacy_old')*perr,fmt='b*',markersize=7,elinewidth=1,capsize=3,label='exp')

                if xl_compare == True and cond.area_avg_ai_sheet != '#REF!':
                    plt.figure(i)
                    plt.subplot(221)
                    plt.errorbar(cond.LoverD,cond.area_avg_ai_sheet,cond.area_avg_ai_sheet*perr,fmt='rx',markersize=7,elinewidth=1,capsize=3,label='exp')

                plt.figure(i)
                plt.subplot(222)

                plt.figure(i)
                plt.subplot(223)
                plt.errorbar(cond.LoverD,cond.area_avg('alpha',method='legacy_old'),cond.area_avg('alpha',method='legacy_old')*perr,fmt='b*',markersize=7,elinewidth=1,capsize=3)

                if xl_compare == True and cond.area_avg_void_sheet != '#REF!':
                    plt.figure(i)
                    plt.subplot(223)
                    plt.errorbar(cond.LoverD,cond.area_avg_void_sheet,cond.area_avg_void_sheet*perr,fmt='rx',markersize=7,elinewidth=1,capsize=3)

                p = (cond.jgatm * 101325 / cond.jgloc)

                plt.figure(i)
                plt.subplot(224)
                plt.errorbar(cond.LoverD,p,p*perr,fmt='b*',markersize=7,elinewidth=1,capsize=3)

                if xl_compare == True:
                    wb = op.load_workbook(filename="H:\TRSL-H\IATE\Legacy\Talley_Suite\H\standard_talley.xlsx", data_only=True)

                    ws = wb['p_exp']
                    talley_p_exp = np.zeros((9,3))
                    for ridx, row in enumerate(ws.iter_rows(min_row=1,max_row=9,values_only=True)):
                        for cidx, value in enumerate(row):
                            talley_p_exp[ridx][cidx] = value

                    plt.figure(i)
                    plt.subplot(224)
                    plt.errorbar(cond.LoverD,talley_p_exp[i][ii],talley_p_exp[i][ii]*perr,fmt='rx',markersize=7,elinewidth=1,capsize=3)

                for iii, cp in enumerate(endpoints):
                    plt.figure(i)

                    plt.subplot(221)
                    plt.axvline(x=endpoints[iii],linestyle='-',linewidth=0.5,c='k')
                    plt.subplot(222)
                    plt.axvline(x=endpoints[iii],linestyle='-',linewidth=0.5,c='k')
                    plt.subplot(223)
                    plt.axvline(x=endpoints[iii],linestyle='-',linewidth=0.5,c='k')
                    plt.subplot(224)
                    plt.axvline(x=endpoints[iii],linestyle='-',linewidth=0.5,c='k')

                plt.tight_layout()
            
            else:
                plt.figure(i)
                plt.errorbar(cond.LoverD,cond.area_avg('ai',method='legacy_old'),cond.area_avg('ai',method='legacy_old')*perr,fmt='b*',markersize=7,elinewidth=1,capsize=3,label='exp')

                if xl_compare == True and cond.area_avg_ai_sheet != '#REF!':
                    plt.figure(i)
                    plt.errorbar(cond.LoverD,cond.area_avg_ai_sheet,cond.area_avg_ai_sheet*perr,fmt='rx',markersize=7,elinewidth=1,capsize=3,label='exp')

                for iii, cp in enumerate(endpoints):
                    plt.figure(i)
                    plt.axvline(x=endpoints[iii],linestyle='-',linewidth=0.5,c='k')

                plt.tight_layout()
                
            #######################################################################################################################################################################
            # Compare against Excel standard
            if True:
                wb = op.load_workbook(filename="H:\TRSL-H\IATE\Legacy\Talley_Suite\H\standard_talley.xlsx", data_only=True)

                ws = wb['z']
                talley_z = []
                for row in ws.iter_rows(min_row=1,max_row=1,values_only=True):
                    for value in row:
                        talley_z.append(value)

                ws = wb['ai']
                talley_ai = np.zeros((9,len(talley_z)))
                for ridx, row in enumerate(ws.iter_rows(min_row=1,max_row=9,values_only=True)):
                    for cidx, value in enumerate(row):
                        talley_ai[ridx][cidx] = value

                ws = wb['alpha']
                talley_alpha = np.zeros((9,len(talley_z)))
                for ridx, row in enumerate(ws.iter_rows(min_row=1,max_row=9,values_only=True)):
                    for cidx, value in enumerate(row):
                        talley_alpha[ridx][cidx] = value
                
                ws = wb['pressure']
                talley_pressure = np.zeros((9,len(talley_z)))
                for ridx, row in enumerate(ws.iter_rows(min_row=1,max_row=9,values_only=True)):
                    for cidx, value in enumerate(row):
                        talley_pressure[ridx][cidx] = value

                plt.figure(i)
                ls = '-' #'dotted'#(0,(3,5))
                lw = 1
                lc = 'r'

                if multiplot:
                    plt.subplot(221)
                plt.plot(talley_z,talley_ai[i],lc,linestyle=ls,linewidth=lw)
                
                if multiplot:
                    plt.subplot(223)
                    plt.plot(talley_z,talley_alpha[i],lc,linestyle=ls,linewidth=lw)

                    plt.subplot(224)
                    plt.plot(talley_z,talley_pressure[i],lc,linestyle=ls,linewidth=lw)

                # if not multiplot:
                #     plt.xlim(left = 0, right = 300)
                #     plt.ylim(bottom = 0, top = 300)

                wb.close()

In [ ]:
print(f"\tCOV_RC\t\t\t\tCOV_TI")
print(f"P1\tP2\tP3\t\tP1\tP2\tP3")

p_atm = 101353

for i in range(np.shape(condor)[0]):
    for ii in range(np.shape(condor)[1]):
        cond = condor[i][ii]

        print(f"{round(cond.COV_RC,3)}\t",end='')

    print(f"\t",end='')
    for ii in range(np.shape(condor)[1]):
        cond = condor[i][ii]

        print(f"{round(cond.COV_TI,3)}\t",end='')

    print(f"\n",end='')

print(f"\n",end='')

print(f"\t\tPressure")
print(f"P1\t\tP2\t\tP3")

for i in range(np.shape(condor)[0]):
    for ii in range(np.shape(condor)[1]):
        cond = condor[i][ii]

        print(f"{round(cond.jgatm * p_atm / cond.jgloc ,3)}\t",end='')

    print(f"\n",end='')

In [10]:
# for cond in database:
#     cond:mg.Condition

#     if cond.LoverD == 244:
#         cond.mirror(sym90=True)
#         cond.calc_COV_RC(reconstruct_flag = True)
#         cond.calc_COV_TI(reconstruct_flag = True)

#         if True:
#             title = f"${cond.theta}^o$, $j_f$ = {cond.jf} [m/s], $j_g$ = {cond.jgref} [m/s], {cond.LoverD}D"

#             # print(f"error: {cond.calc_errors('alpha','alpha_reconstructed')}")
#             cond.plot_contour(param = 'alpha', num_levels = 100, colormap = 'jet', cartesian = False)
#             cond.plot_contour(param = 'alpha_reconstructed', num_levels = 100, colormap = 'jet', cartesian = False, plot_measured_points = False)
#             X = cond.plot_profiles2(param = 'alpha_reconstructed', title = title, legend_loc = 'lower right', label_str = "$<α>_{reconstructed}$")

In [ ]:
for cond in database:
    cond:mg.Condition

    if cond.jf == 3.98 and cond.jgref == 0.15 and cond.LoverD == 244:
        wb = op.load_workbook(filename="H:\TRSL-H\IATE\Legacy\Talley_Suite\H\standard_talley.xlsx", data_only=True)
        ws = wb['void_reconstruct_2']

        talley_rec = np.zeros((41,5))
        for ridx, row in enumerate(ws.iter_rows(min_row=1,max_row=41,values_only=True)):
            for cidx, value in enumerate(row):
                talley_rec[ridx][cidx] = value

        fig = plt.figure()
        ax1 = fig.add_subplot(111)

        ls = 'dashed'
        lw = 3
        mkr = ''

        r_points = np.arange(-1,1.05,0.05)
        r_points = r_points[::-1]

        ax1.plot(r_points,talley_rec[:,0],color='r',marker=mkr,linestyle=ls,linewidth=lw)
        ax1.plot(r_points,talley_rec[:,1],color='b',marker=mkr,linestyle=ls,linewidth=lw)
        ax1.plot(r_points,talley_rec[:,2],color='lime',marker=mkr,linestyle=ls,linewidth=lw)
        ax1.plot(r_points,talley_rec[:,3],color='k',marker=mkr,linestyle=ls,linewidth=lw)
        ax1.plot(r_points,talley_rec[:,4],color='gray',marker=mkr,linestyle=ls,linewidth=lw)

        ls = 'solid'
        lw = 1.5
        mkr = ''

        alpha_rec = []
        i = 0
        cs = ['r','b','lime','k','gray']
        for angle, r_dict in cond.data.items():
            if angle <= 90:
                rs = []
                vals = []

                for rstar, midas_dict in r_dict.items():
                    if rstar >= 0:
                        rs.append(rstar)
                        vals.append(midas_dict['alpha_reconstructed'])

                        print(f"angle: {angle} \tr*: {rstar} \talpha: {midas_dict['alpha_reconstructed']}")
                    
                vals = [var for _, var in sorted(zip(rs, vals))]
                rs = sorted(rs)

                ax1.plot(rs,vals,color=cs[i],marker=mkr,linestyle=ls,linewidth=lw)
                plt.xlim(0,1)
                plt.xlabel('r/R [-]')
                plt.ylabel(r'$\alpha$ [-]')
                i = i + 1

        wb.close()
